In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'han_auto'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

#ddday = 31
#mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    #df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
#                               colum_idx[idx], interpolation=interpolation_option[idx],
#                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2015.xlsx', '가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx', '가평_2019.xlsx', '가평_2020.xlsx'], ['서상_2015.xlsx', '서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx', '서상_2019.xlsx', '서상_2020.xlsx'], ['의암호_2015.xlsx', '의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx', '의암호_2019.xlsx', '의암호_2020.xlsx']]
data/han/자동/가평_2015.xlsx
data/han/자동/가평_2016.xlsx
data/han/자동/가평_2017.xlsx
data/han/자동/가평_2018.xlsx
data/han/자동/가평_2019.xlsx
data/han/자동/가평_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/han/자동/서상_2015.xlsx
data/han/자동/서상_2016.xlsx
data/han/자동/서상_2017.xlsx
data/han/자동/서상_2018.xlsx
data/han/자동/서상_2019.xlsx
data/han/자동/서상_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/han/자동/의암호_2015.xlsx
data/han/자동/의암호_2016.xlsx
data/han/자

1/1 [==============================] - 0s 20ms/step - gen_loss: 10.8079 - disc_loss: 0.3478 - rmse: 0.3842 - val_loss: 0.3983
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.5179 - disc_loss: 0.3572 - rmse: 0.3873 - val_loss: 0.3418
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.5740 - disc_loss: 0.3576 - rmse: 0.4132 - val_loss: 0.3716
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.3841 - disc_loss: 0.3358 - rmse: 0.3837 - val_loss: 0.4090
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.6108 - disc_loss: 0.3359 - rmse: 0.3906 - val_loss: 0.4788
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.5832 - disc_loss: 0.3615 - rmse: 0.4004 - val_loss: 0.3648
Epoch 56/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.2653 - disc_loss: 0.3537 - rmse: 0.3718 - val_loss: 0.3711
Epoch 57/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 9.7653 - disc_loss: 0.2883 - rmse: 0.3997 - val_loss: 0.3728
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0081 - disc_loss: 0.2768 - rmse: 0.3171 - val_loss: 0.4389
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.1270 - disc_loss: 0.2964 - rmse: 0.3722 - val_loss: 0.3323
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.2202 - disc_loss: 0.2688 - rmse: 0.4089 - val_loss: 0.4060
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2530 - disc_loss: 0.2904 - rmse: 0.3473 - val_loss: 0.3320
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2372 - disc_loss: 0.2835 - rmse: 0.3640 - val_loss: 0.3756
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2139 - disc_loss: 0.2700 - rmse: 0.3202 - val_loss: 0.3621
Epoch 115/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6847 - disc_loss: 0.2712 - rmse: 0.2809 - val_loss: 0.4166
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.4794 - disc_loss: 0.2608 - rmse: 0.3654 - val_loss: 0.3763
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1956 - disc_loss: 0.2634 - rmse: 0.3117 - val_loss: 0.3045
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5036 - disc_loss: 0.2630 - rmse: 0.3274 - val_loss: 0.3833
Epoch 170/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.2521 - disc_loss: 0.2661 - rmse: 0.3172 - val_loss: 0.3780
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.9057 - disc_loss: 0.2608 - rmse: 0.2952 - val_loss: 0.2976
Epoch 172/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.7382 - disc_loss: 0.2562 - rmse: 0.3377 - val_loss: 0.4116
Epoch 173/2000
1/1 [===============

1/1 [==============================] - 0s 21ms/step - gen_loss: 7.5587 - disc_loss: 0.2452 - rmse: 0.3369 - val_loss: 0.3540
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7915 - disc_loss: 0.2533 - rmse: 0.2784 - val_loss: 0.3231
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2957 - disc_loss: 0.2179 - rmse: 0.3167 - val_loss: 0.3697
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8638 - disc_loss: 0.2295 - rmse: 0.3468 - val_loss: 0.4280
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1119 - disc_loss: 0.2438 - rmse: 0.3117 - val_loss: 0.3904
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1583 - disc_loss: 0.2432 - rmse: 0.3192 - val_loss: 0.3492
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5956 - disc_loss: 0.2201 - rmse: 0.3022 - val_loss: 0.3164
Epoch 231/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4044 - disc_loss: 0.1814 - rmse: 0.2770 - val_loss: 0.2721
Epoch 1385/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0373 - disc_loss: 0.1622 - rmse: 0.2432 - val_loss: 0.3269
Epoch 1386/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3393 - disc_loss: 0.1780 - rmse: 0.2565 - val_loss: 0.3374
Epoch 1387/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3184 - disc_loss: 0.1703 - rmse: 0.2726 - val_loss: 0.2777
Epoch 1388/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9636 - disc_loss: 0.1645 - rmse: 0.2357 - val_loss: 0.3048
Epoch 1389/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.5976 - disc_loss: 0.1673 - rmse: 0.3049 - val_loss: 0.3089
Epoch 1390/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6394 - disc_loss: 0.1801 - rmse: 0.2446 - val_loss: 0.2842
Epoch 1391/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3404 - disc_loss: 0.1636 - rmse: 0.2984 - val_loss: 0.3426
Epoch 1443/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8967 - disc_loss: 0.1556 - rmse: 0.2495 - val_loss: 0.2741
Epoch 1444/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7112 - disc_loss: 0.1660 - rmse: 0.2395 - val_loss: 0.2792
Epoch 1445/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4625 - disc_loss: 0.1727 - rmse: 0.3054 - val_loss: 0.3577
Epoch 1446/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8566 - disc_loss: 0.1733 - rmse: 0.2435 - val_loss: 0.2790
Epoch 1447/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0584 - disc_loss: 0.1730 - rmse: 0.2789 - val_loss: 0.3402
Epoch 1448/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2678 - disc_loss: 0.1701 - rmse: 0.2585 - val_loss: 0.2808
Epoch 1449/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1561 - disc_loss: 0.1650 - rmse: 0.8254 - val_loss: 0.2717
Epoch 1501/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0716 - disc_loss: 0.1753 - rmse: 0.2444 - val_loss: 0.3247
Epoch 1502/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6820 - disc_loss: 0.1728 - rmse: 0.2501 - val_loss: 0.2904
Epoch 1503/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.5314 - disc_loss: 0.1534 - rmse: 0.3327 - val_loss: 0.4207
Epoch 1504/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3438 - disc_loss: 0.1798 - rmse: 0.2856 - val_loss: 0.2558
Epoch 1505/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7838 - disc_loss: 0.1596 - rmse: 0.2461 - val_loss: 0.3259
Epoch 1506/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5330 - disc_loss: 0.1664 - rmse: 0.2618 - val_loss: 0.2787
Epoch 1507/2000
1/1 [========

1/1 [==============================] - 0s 21ms/step - gen_loss: 5.7462 - disc_loss: 0.1703 - rmse: 0.3421 - val_loss: 0.2607
Epoch 1559/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3869 - disc_loss: 0.1627 - rmse: 0.3361 - val_loss: 0.4086
Epoch 1560/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9483 - disc_loss: 0.1657 - rmse: 0.2837 - val_loss: 0.2759
Epoch 1561/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5871 - disc_loss: 0.1664 - rmse: 0.3526 - val_loss: 0.3539
Epoch 1562/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4035 - disc_loss: 0.1713 - rmse: 0.3469 - val_loss: 0.3538
Epoch 1563/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7647 - disc_loss: 0.1671 - rmse: 0.2696 - val_loss: 0.2654
Epoch 1564/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4530 - disc_loss: 0.1585 - rmse: 0.2382 - val_loss: 0.2909
Epoch 1565/2000
1/1 [========

1/1 [==============================] - 0s 22ms/step - gen_loss: 5.4961 - disc_loss: 0.1710 - rmse: 0.3133 - val_loss: 0.3559
Epoch 1617/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7256 - disc_loss: 0.1564 - rmse: 0.2769 - val_loss: 0.3000
Epoch 1618/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1218 - disc_loss: 0.1539 - rmse: 0.2817 - val_loss: 0.2494
Epoch 1619/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3164 - disc_loss: 0.1594 - rmse: 0.2429 - val_loss: 0.4421
Epoch 1620/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5780 - disc_loss: 0.1569 - rmse: 0.2608 - val_loss: 0.2606
MissData :  save/  miss :  (7071, 12)
1/1 [==============================] - 0s 4ms/step - loss: 0.2619
MissData :  save/  miss :  (7071, 12)


In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [32]:
print(real_df_all.shape)
rnn_target_column = 'toc'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(52608, 36)
toc


In [33]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (36825, 36) val_df.shape :  (10522, 36) test_df.shape: (5261, 36)


In [34]:
#fake_df.shape[0]/8760

In [35]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [38]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  4
out_num_features :  1


In [39]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [40]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/han/models/han_auto/toc/


In [41]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [42]:
rnn_epochs

20

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.5336 - mean_absolute_error: 0.5293 - nse: 0.1748
Epoch 00001: val_nse improved from -inf to 0.24260, saving model to save/best_model.h5
10/10 [==============================] - 8s 815ms/step - loss: 0.5336 - mean_absolute_error: 0.5293 - nse: 0.1748 - val_loss: 0.1639 - val_mean_absolute_error: 0.3354 - val_nse: 0.2426
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.3031 - mean_absolute_error: 0.3889 - nse: 0.5087
Epoch 00002: val_nse improved from 0.24260 to 0.39830, saving model to save/best_model.h5
10/10 [==============================] - 9s 862ms/step - loss: 0.3031 - mean_absolute_error: 0.3889 - nse: 0.5087 - val_loss: 0.1304 - val_mean_absolute_error: 0.2874 - val_nse: 0.3983
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.2136 - mean_absolute_error: 0.3206 - nse: 0.6512
Epoch 00003: val_nse improved from 0.39830 to 0.45470, saving model to save/best_model.h5
10/10 

Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.1617 - mean_absolute_error: 0.2619 - nse: 0.7485
Epoch 00023: val_nse did not improve from 0.59113
10/10 [==============================] - 8s 785ms/step - loss: 0.1617 - mean_absolute_error: 0.2619 - nse: 0.7485 - val_loss: 0.2646 - val_mean_absolute_error: 0.3669 - val_nse: -0.2202
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.1534 - mean_absolute_error: 0.2550 - nse: 0.7618
Epoch 00024: val_nse did not improve from 0.59113
10/10 [==============================] - 8s 766ms/step - loss: 0.1534 - mean_absolute_error: 0.2550 - nse: 0.7618 - val_loss: 0.2027 - val_mean_absolute_error: 0.3229 - val_nse: 0.0627
Epoch 25/50
10/10 [==============================] - ETA: 0s - loss: 0.1494 - mean_absolute_error: 0.2536 - nse: 0.7568
Epoch 00025: val_nse did not improve from 0.59113
10/10 [==============================] - 8s 790ms/step - loss: 0.1494 - mean_absolute_error: 0.2536 - nse: 0.7568 - val

10/10 [==============================] - ETA: 0s - loss: 0.1301 - mean_absolute_error: 0.2302 - nse: 0.7961
Epoch 00046: val_nse did not improve from 0.59113
10/10 [==============================] - 8s 759ms/step - loss: 0.1301 - mean_absolute_error: 0.2302 - nse: 0.7961 - val_loss: 0.3012 - val_mean_absolute_error: 0.3753 - val_nse: -0.3929
Epoch 47/50
10/10 [==============================] - ETA: 0s - loss: 0.1241 - mean_absolute_error: 0.2269 - nse: 0.8100
Epoch 00047: val_nse did not improve from 0.59113
10/10 [==============================] - 8s 779ms/step - loss: 0.1241 - mean_absolute_error: 0.2269 - nse: 0.8100 - val_loss: 0.3288 - val_mean_absolute_error: 0.3872 - val_nse: -0.5252
Epoch 48/50
10/10 [==============================] - ETA: 0s - loss: 0.1114 - mean_absolute_error: 0.2250 - nse: 0.8215
Epoch 00048: val_nse did not improve from 0.59113
10/10 [==============================] - 8s 770ms/step - loss: 0.1114 - mean_absolute_error: 0.2250 - nse: 0.8215 - val_loss: 0.27

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [29]:
#train_df.shape, val_df.shape, test_df.shape

In [30]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
